### Compute TPSim by odor OR cond
for Encoding OR Retrieval separately

In [11]:
from brainpipe.system import study
from itertools import combinations, product
import numpy as np
from os.path import join, exists
from os import makedirs
from joblib import Parallel, delayed
from scipy import stats

In [3]:
"""
Compute TPSim BY ODOR
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
path2save = join(st.path, 'feature/TPSim_3groups_{}/by_odor_wth_v=1_elecs=all/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
savename = 'TPS_pears_{}_odor{}_wth_{}.npz'
###############################################################################
reps = ['Enc'] #Ret, Enc
for rep in reps:
    if not exists(path2save.format(rep)):
        makedirs(path2save.format(rep))
###############################################################################
#freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
freqs = ['delta','theta','alpha','beta','low_gamma','high_gamma'] # Name of each frequency
conds = ['high','low']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

def tpsim_by_odor(exp,su,odor,f,freq):
    mat = np.load(pow_path.format(su,odor,rep[0]),allow_pickle=True)
    pow_data = mat['xpow'][f,:,to_take[0]:to_take[1],:]
    print(su,odor,pow_data.shape)
    nelecs,npts,ntrials = pow_data.shape
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(nelecs):
        pow0 = pow_data[elec,:]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in combinations(np.arange(ntrials), 2):
            R, p = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,odor,freq,'R',R_freq.shape, p_freq.shape,'initial data',pow_data.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat['labels'],
           'channel':mat['channels'], 'xyz':mat['xyz']}
    np.savez(path2save.format(exp)+savename.format(su,str(odor),freq),**mat)

for rep in reps:
    for su in ['PIRJ']:
        for f, freq in enumerate(freqs):
            for odor in odors_su[su]:
                tpsim_by_odor(rep,su,odor,f,freq)
#     Parallel(n_jobs=-1)(delayed(
#         tpsim_by_odor)(su,odor,freq) for odor,freq in product(odors_su[su],freqs))

-> Olfacto loaded
PIRJ 4 (24, 30, 4)
PIRJ 4 delta R (24, 6) (24, 6) initial data (24, 30, 4)
PIRJ 9 (24, 30, 6)
PIRJ 9 delta R (24, 15) (24, 15) initial data (24, 30, 6)
PIRJ 1 (24, 30, 7)
PIRJ 1 delta R (24, 21) (24, 21) initial data (24, 30, 7)
PIRJ 18 (24, 30, 5)
PIRJ 18 delta R (24, 10) (24, 10) initial data (24, 30, 5)
PIRJ 6 (24, 30, 1)
PIRJ 6 delta R (0,) (0,) initial data (24, 30, 1)
PIRJ 5 (24, 30, 1)
PIRJ 5 delta R (0,) (0,) initial data (24, 30, 1)
PIRJ 7 (24, 30, 1)
PIRJ 7 delta R (0,) (0,) initial data (24, 30, 1)
PIRJ 4 (24, 30, 4)
PIRJ 4 theta R (24, 6) (24, 6) initial data (24, 30, 4)
PIRJ 9 (24, 30, 6)
PIRJ 9 theta R (24, 15) (24, 15) initial data (24, 30, 6)
PIRJ 1 (24, 30, 7)
PIRJ 1 theta R (24, 21) (24, 21) initial data (24, 30, 7)
PIRJ 18 (24, 30, 5)
PIRJ 18 theta R (24, 10) (24, 10) initial data (24, 30, 5)
PIRJ 6 (24, 30, 1)
PIRJ 6 theta R (0,) (0,) initial data (24, 30, 1)
PIRJ 5 (24, 30, 1)
PIRJ 5 theta R (0,) (0,) initial data (24, 30, 1)
PIRJ 7 (24, 30, 1)
PI

In [4]:
"""
Compute TPSim BTW odors By ODOR
>>> in order to compute dissimilarity matrices OR compare WTH / BTW
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/by_odor_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_pears_{}_odor{}_btw_{}.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
conds = ['high','low']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

def tpsim_btw_odors_by_odor(su,o1,o2,f,freq):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    su : string (subject name)
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2)
    """
    mat_o1 = np.load(pow_path.format(su,o1,shortcut[0]),allow_pickle=True)
    print(mat_o1.files)
    pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
    pow_o2 = np.array([])
    for o in o2:
        pow_ = np.load(pow_path.format(su,o,shortcut[0],freq))['xpow'][f,:,17:47,:]
        pow_o2 = np.concatenate((pow_o2,pow_),axis=-1) if np.size(pow_o2) else pow_
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.pearsonr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,o1,freq,'R',R_freq.shape, p_freq.shape,
              'o1 data',pow_o1.shape, 'o2 data',pow_o2.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat_o1['labels'],
           'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
    np.savez(tps_save.format(exp,su,o1,freq),**mat)
    
for su in ['PIRJ']:
    for o1, freq in product(odors_su[su],freqs):
        f = [i for i,f in enumerate(freqs) if f == freq][0]
        o2 = [o for o in odors_su[su] if o != o1]
        tpsim_btw_odors_by_odor(su,o1,o2,f,freq)

-> Olfacto loaded
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 delta R (24, 84) (24, 84) o1 data (24, 30, 4) o2 data (24, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 theta R (24, 84) (24, 84) o1 data (24, 30, 4) o2 data (24, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 alpha R (24, 84) (24, 84) o1 data (24, 30, 4) o2 data (24, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 beta R (24, 84) (24, 84) o1 data (24, 30, 4) o2 data (24, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 low_gamma R (24, 84) (24, 84) o1 data (24, 30, 4) o2 data (24, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 high_gam

In [3]:
"""
Compute TPSim BTW CONTEXT By ODOR
computing TPSim btw 2 lists of odors
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path,'feature/TPSim_3groups_{}/by_cont_{}_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_pears_{}_odor{}_{}_{}.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
methods = ['wth','btw']
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0],exp[0]]
###############################################################################
freqnames = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freqs = ['theta']
context_su = {'CHAF': {'M':[5,7,8,9],'F':[1,2,3,4]},
              'LEFC': {'M':[1,2,3,4],'F':[14,15,16,17]},
              'PIRJ': {'M':[4,9,1,18],'F':[6,5,7]}, #missing odor 15
              'VACJ': {'M':[14,15,16,17],'F':[10,11,12,13]},
              'SEMC': {'M':[10,11,12,13],'F':[5,7,8,9]},
              'FERJ': {'M':[16,17,5,7],'F':[12,13,2,1]},}

def tpsim_by_odor(su,o1,o2,f,freq,meth):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    su : string (subject name)
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2
    """
    mat_o1 = np.load(pow_path.format(su,o1,shortcut[0]),allow_pickle=True)
    pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
    pow_o2 = np.array([])
    for o in o2:
        pow_ = np.load(pow_path.format(su,o,shortcut[1]))['xpow'][f,:,17:47,:]
        pow_o2 = np.concatenate((pow_o2,pow_),axis=-1) if np.size(pow_o2) else pow_
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.pearsonr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,o1,freq,'R',R_freq.shape, p_freq.shape,
              'o1 data',pow_o1.shape, 'o2 data',pow_o2.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat_o1['labels'],
           'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
    np.savez(tps_save.format(exp,meth,su,o1,meth,freq),**mat)

for meth in methods:
    if not exists(path2save.format(exp,meth)):
        makedirs(path2save.format(exp,meth))
    for su in context_su:
        for cont in context_su[su]:
            for o1, freq in product(context_su[su][cont],freqs):
                f = [i for i,f in enumerate(freqnames) if f == freq][0]
                if meth == 'wth':
                    o2 = [o for o in context_su[su][cont] if o != o1] #wth tps
                if meth == 'btw':
                    oth_cont = [c for c in context_su[su] if c!= cont][0]
                    o2 = context_su[su][oth_cont] #btw tps
                tpsim_by_odor(su,o1,o2,f,freq,meth)

-> Olfacto loaded
CHAF 5 theta R (61, 12) (61, 12) o1 data (61, 30, 1) o2 data (61, 30, 12)
CHAF 7 theta R (61, 36) (61, 36) o1 data (61, 30, 4) o2 data (61, 30, 9)
CHAF 8 theta R (61, 30) (61, 30) o1 data (61, 30, 3) o2 data (61, 30, 10)
CHAF 9 theta R (61, 40) (61, 40) o1 data (61, 30, 5) o2 data (61, 30, 8)
CHAF 1 theta R (61, 7) (61, 7) o1 data (61, 30, 1) o2 data (61, 30, 7)
CHAF 2 theta R (61, 15) (61, 15) o1 data (61, 30, 3) o2 data (61, 30, 5)
CHAF 3 theta R (61, 15) (61, 15) o1 data (61, 30, 3) o2 data (61, 30, 5)
CHAF 4 theta R (61, 7) (61, 7) o1 data (61, 30, 1) o2 data (61, 30, 7)
LEFC 1 theta R (27, 26) (27, 26) o1 data (27, 30, 2) o2 data (27, 30, 13)
LEFC 2 theta R (27, 50) (27, 50) o1 data (27, 30, 5) o2 data (27, 30, 10)
LEFC 3 theta R (27, 44) (27, 44) o1 data (27, 30, 4) o2 data (27, 30, 11)
LEFC 4 theta R (27, 44) (27, 44) o1 data (27, 30, 4) o2 data (27, 30, 11)
LEFC 14 theta R (27, 204) (27, 204) o1 data (27, 30, 6) o2 data (27, 30, 34)
LEFC 15 theta R (27, 319) (

In [ ]:
"""
Compute TPSim BTW CONTEXT BTW SPACE
computing TPSim btw 2 lists of odors
Positions groups: (1odor by context, grouped by closest positions)
Gr1 1/5 Gr2 6/3 Gr3 7/2 Gr4 8/4
"""

###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_path = join(path_pow, '{}_odor_{}_{}_6freqs.npz')
path2save = join(st.path,'feature/TPSim_3groups_{}/by_pos_{}/')
tps_save = join(path2save, 'TPS_pears_{}_odor{}_{}_{}.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
methods = ['wth','btw']
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0],exp[0]]
###############################################################################
freqnames = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freqs = ['theta']
od_to_pos = {'CHAF': {'P1':[1,5],'P2':[7,3],'P3':[8,2], 'P4':[4,9]},
              'LEFC': {'P1':[14,1],'P2':[2,16],'P3':[3,15], 'P4':[4,17]},
              'PIRJ': {'P1':[6,4],'P2':[9],'P3':[1,5], 'P4':[18,7]}, #missing odor 15
              'VACJ': {'P1':[10,14],'P2':[15,12],'P3':[16,11], 'P4':[17,13]},
              'SEMC': {'P1':[5,10],'P2':[11,8],'P3':[12,7], 'P4':[13,9]},
              'FERJ': {'P1':[12,16],'P2':[17,2],'P3':[5,13], 'P4':[7,1]},}

def tpsim_by_odor(su,o1,o2,f,freq,meth):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    su : string (subject name)
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2
    """
    mat_o1 = np.load(pow_path.format(su,o1,shortcut[0]),allow_pickle=True)
    pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
    pow_o2 = np.array([])
    for o in o2:
        pow_ = np.load(pow_path.format(su,o,shortcut[1]))['xpow'][f,:,17:47,:]
        pow_o2 = np.concatenate((pow_o2,pow_),axis=-1) if np.size(pow_o2) else pow_
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.spearmanr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,o1,freq,'R',R_freq.shape, p_freq.shape,
              'o1 data',pow_o1.shape, 'o2 data',pow_o2.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat_o1['labels'],
           'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
    np.savez(tps_save.format(exp,meth,su,o1,meth,freq),**mat)

for meth in methods:
    if not exists(path2save.format(exp,meth)):
        makedirs(path2save.format(exp,meth))
    for su in od_to_pos:
        for pos in od_to_pos[su]:
            for o1, freq in product(od_to_pos[su][pos],freqs):
                f = [i for i,f in enumerate(freqnames) if f == freq][0]
                if meth == 'wth':
                    o2 = [o for o in od_to_pos[su][pos] if o != o1] #wth tps
                if meth == 'btw':
                    oth_cont = [c for c in od_to_pos[su] if c!= pos]
                    o2 = np.ravel([od_to_pos[su][p] for p in oth_cont])
                tpsim_by_odor(su,o1,o2,f,freq,meth)

### Distance between circles at encoding
- Compute distances between circles
- Create close and far-away groups for each position
- Create link btw odors and position for each patient

In [ ]:
from utils import pos_cirles
from itertools import combinations
from codes import num_odor_to_ref
import pandas as pd

#compute all distances between circles
p1_, p2_, dist_ = [], [], []
for p1, p2 in combinations(pos_cirles,2):
    p1_.append(p1), p2_.append(p2)
    coord1, coord2 = np.array(pos_cirles[p1]), np.array(pos_cirles[p2])
    dist = np.round(np.linalg.norm(coord1-coord2),2)
    dist_.append(dist)
data = np.concatenate((np.array(p1_)[:,np.newaxis],np.array(p2_)[:,np.newaxis],
                       np.array(dist_,dtype='float')[:,np.newaxis]),axis=1)
df = pd.DataFrame(data,columns=['Pos1','Pos2','dist'])
df = df.astype({'dist': 'float'})

#create a group of close and away circles for each place
dict_pos_dist = {}
for pos in pos_cirles:
    dict_inter = {}
    df_sel = df.loc[(df['Pos1']==pos) | (df['Pos2']==pos)].sort_values('dist',ascending=False)
    #print(df_sel)
    pos_sorted = df_sel[['Pos1']].values[:,0]+df_sel[['Pos2']].values[:,0]
    pos_sorted = [p[:2] if p[:2] != pos else p[-2:] for p in pos_sorted ]
    dict_inter['close'] = pos_sorted[-1:]
    dict_inter['far'] = pos_sorted[:1]
    dict_pos_dist[pos] = dict_inter
print(dict_pos_dist)

dict_pos_od = {}
for su in num_odor_to_ref:
    dict_pos_su = {}
    for od in num_odor_to_ref[su]:
        dict_pos_su['P'+str(num_odor_to_ref[su][od][1])] = str(od)
    dict_pos_od[su] = dict_pos_su
del dict_pos_od['MICP']
print(dict_pos_od)

In [ ]:
"""
Compute TPSim BTW SPACE LOCATIONS
computing TPSim btw 2 lists of odors
Positions groups: (1odor by context, grouped by closest positions)
"""

###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_path = join(path_pow, '{}_odor_{}_{}_6freqs.npz')
path2save = join(st.path,'feature/TPSim_3groups_{}/by_pos_{}/')
tps_save = join(path2save, 'TPS_pears_{}_odor{}_{}_{}.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
methods = ['wth','btw']
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0],exp[0]]
###############################################################################
freqnames = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freq = 'alpha'
f = [i for i,f in enumerate(freqnames) if f == freq][0]

def tpsim_by_odor(su,o1,o2,f,freq,meth):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    su : string (subject name)
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2
    """
    mat_o1 = np.load(pow_path.format(su,o1,shortcut[0]),allow_pickle=True)
    pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
    pow_o2 = np.array([])
    for o in o2:
        pow_ = np.load(pow_path.format(su,o,shortcut[1]))['xpow'][f,:,17:47,:]
        pow_o2 = np.concatenate((pow_o2,pow_),axis=-1) if np.size(pow_o2) else pow_
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.spearmanr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,o1,freq,'R',R_freq.shape, p_freq.shape,
              'o1 data',pow_o1.shape, 'o2 data',pow_o2.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat_o1['labels'],
           'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
    np.savez(tps_save.format(exp,meth,su,o1,meth,freq),**mat)

for meth in methods:
    if not exists(path2save.format(exp,meth)):
        makedirs(path2save.format(exp,meth))
    for su in dict_pos_od:
        for pos in dict_pos_dist:
            if su == 'PIRJ' and pos == 'P3':
                continue
            else:
                print(pos, su)
                o1 = dict_pos_od[su][pos]
                if meth == 'wth':
                    pos_list = dict_pos_dist[pos]['close']
                    if su == 'PIRJ':
                        pos_list = [p for p in pos_list if p != 'P3']
                    od_list = [dict_pos_od[su][p] for p in pos_list]
                    print(su,meth,pos,pos_list,od_list)
                if meth == 'btw':
                    pos_list = dict_pos_dist[pos]['far']
                    if su == 'PIRJ':
                        if pos not in ['P5','P6']:
                            pos_list = [p for p in pos_list if p != 'P3']
                        if pos == 'P5':
                            pos_list = ['P4']
                        if pos == 'P6':
                            pos_list = ['P2']
                    od_list = [dict_pos_od[su][p] for p in pos_list]
                    print(su,meth,pos,pos_list,od_list)
                tpsim_by_odor(su,o1,od_list,f,freq,meth)

In [ ]:
"""
Combine TPSim files of each odor BY CONDITION
"""
from utils import odor_REC_first_3wgth as dict_
RT_type = 'RT_rec_first'#,'RT_epi_wght','RT_rec_first','RT_rec_wght','RT_epi_first
###############################################################################
st = study('Olfacto')
path_tps = join(st.path,'feature/TPSim_{}_By_Odor_By_Cond/TPS_by_odor/by_odor_wth/')
tps_file = join(path_tps, 'TPS_pears_{}_odor_{}_{}.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}_RT/TPS_wth_'+RT_type+'/')
save_file = join(path2save, 'TPS_pears_{}_{}_wth_RT_{}.npz')
###############################################################################
freqs = ['delta','theta','alpha','beta','low_gamma','high_gamma']
phases = ['Enc_Ret','Enc','Ret']
    
for su, phase in product(dict_,phases):
    if not exists(path2save.format(phase)):
        makedirs(path2save.format(phase))
    for cond in dict_[su]:
        for freq in freqs:
            concat = np.array([])
            for odor in dict_[su][cond]:
                if su == 'CHAF' and odor not in ['1','4','5'] and phase == 'Enc':
                    data = np.load(tps_file.format(phase,su,odor,freq))
                    tps = data['tps']
                    if len(tps) > 0:
                        concat = np.concatenate((concat,tps), axis=1) if np.size(concat) else tps
                if su == 'PIRJ' and odor not in ['5','6','7'] and phase == 'Enc':
                    data = np.load(tps_file.format(phase,su,odor,freq))
                    tps = data['tps']
                    if len(tps) > 0:
                        concat = np.concatenate((concat,tps), axis=1) if np.size(concat) else tps
                if su not in ['CHAF','PIRJ'] or phase != 'Enc':
                    data = np.load(tps_file.format(phase,su,odor,freq))
                    tps = data['tps']
                    if len(tps) > 0:
                        concat = np.concatenate((concat,tps), axis=1) if np.size(concat) else tps
            print(su, freq, cond, concat.shape)
            mat = {'tps':concat, 'label':data['label'],'channel':data['channel'],
                   'xyz':data['xyz']}
            np.savez(save_file.format(phase,su,cond,freq),**mat)

In [95]:
"""
Compute TPSim by combining all odors from each CONDITION
"""
exp = 'Enc' #Ret, Enc
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
pow_file = join(path_data, '{}_odor_{}_'+exp[0]+'_6freqs_hilbert=ok_ds.npz')
pathsave = join(st.path,'feature/TPSim_3groups_{}/TPS_btw_v=1_elecs=all_hilbert=ok/')#_'+RT_type+'/')
savename = join(pathsave,'TPS_pears_{}_{}_btw_{}_width={}_step={}_ds.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
#print('save path',path2save.format(exp))
#print('pow files',pow_file)
#############################################RT_type##################################
# freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freqs = ['theta']
subjects = ['PIRJ','CHAF','FERJ','VACJ','SEMC','LEFC']
conds = ['low','mid','high']

def tpsim_by_cond(su,cond):
    mat = np.load(pow_file.format(su,cond),allow_pickle=True)
    width, step = mat['width'], mat['step']
    fnames = mat['fname']
    time = mat['time_ds'] - 3
    nsel = [i for i,t in enumerate(time) if -1<=t<=2]
    print(nsel[0],nsel[-1])
    print(su,cond,mat['xpow_down'].shape,np.unique(mat['labels']))
    pow_data = mat['xpow_down'][:,:,nsel,:] #3584 points
    _,nelecs,npts,ntrials = pow_data.shape
    for freq in freqs:
        f = [i for i,f in enumerate(fnames) if f == freq][0]
        data_freq = pow_data[f]
        R_freq, p_freq = np.array([]), np.array([])
        for elec in range(nelecs):
            pow0 = data_freq[elec,...]
            R_trials, p_trials = np.array([]), np.array([])
            for t0, t1 in combinations(np.arange(ntrials), 2):
                R, p = stats.pearsonr(pow0[:,t0],pow0[:,t1])
                D = 1 - R # <<<<<<< HERE TO CHANGE FOR DISTANCE COMPUTATIONS
                R_trials = np.vstack((R_trials,D)) if np.size(R_trials) else D
                p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
            R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
            p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #R_freq = np.arctanh(R_freq)
        print(su,cond,freq,'TPSim',R_freq.shape, p_freq.shape,'initial data',pow_data.shape)
        dict_ = {'tps':R_freq, 'pval':p_freq,'label':mat['labels'],
                           'channel':mat['channels'], 'xyz':mat['xyz']}
        np.savez(savename.format(exp,su,cond,freq,width,step),**dict_)

for su,cond in product(subjects,conds):
    tpsim_by_cond(su,cond)
# Parallel(n_jobs=-1)(delayed(
#     tpsim_by_cond)(su,cond) for su,cond in product(subjects,conds))

-> Olfacto loaded
20 50
PIRJ low (6, 18, 70, 14) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
PIRJ low theta TPSim (18, 91) (18, 91) initial data (6, 18, 31, 14)
20 50
PIRJ mid (6, 18, 70, 9) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
PIRJ mid theta TPSim (18, 36) (18, 36) initial data (6, 18, 31, 9)
20 50
PIRJ high (6, 18, 70, 20) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
PIRJ high theta TPSim (18, 190) (18, 190) initial data (6, 18, 31, 20)
20 50
CHAF low (6, 61, 70, 5) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF low theta TPSim (61, 10) (61, 10) initial data (6, 61, 31, 5)
20 50
CHAF mid (6, 61, 70, 4) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF mid theta TPSim (61, 6) (61, 6) initial data (6, 61, 31, 4)
20 50
CHAF high (6, 61, 70, 12) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF high theta TPSim (61, 66) (61, 66) initial data (6, 61, 31, 12)
20 50
FERJ low (6, 32, 70, 16) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
FERJ low theta TPSim (32, 120) (32, 120) initial data (6, 32, 31, 16)
20 50
FERJ mid (6, 32, 70, 6) ['Amg' 'IFG

In [3]:
"""
Compute TPSim by combining all odors from each CONDITION for Early/Late
"""
exp = 'Enc' #Ret, Enc
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
pow_file = join(path_data, '{}_odor_{}_'+exp[0]+'_6freqs_EL.npz')
pathsave = join(st.path,'feature/TPSim_3groups_{}/TPS_btw_v=1_elecs=all/')#_'+RT_type+'/')
savename = join(pathsave,'TPS_pears_{}_{}_btw_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
#print('save path',path2save.format(exp))
#print('pow files',pow_file)
#############################################RT_type##################################
# freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freqs = ['theta']
subjects = ['PIRJ','CHAF','FERJ','VACJ','SEMC','LEFC']
conds,step = ['low','mid','high'], 'Early'
to_take = [17,47]

def tpsim_by_cond(su,cond):
    mat = np.load(pow_file.format(su,cond),allow_pickle=True)
    print(su,cond,mat['xpow_'+step[0]].shape,np.unique(mat['labels']))
    pow_data = mat['xpow_'+step[0]][:,:,to_take[0]:to_take[1],:] #3584 points 
    _,nelecs,npts,ntrials = pow_data.shape
    for f,freq in enumerate(freqs):
        data_freq = pow_data[f]
        R_freq, p_freq = np.array([]), np.array([])
        for elec in range(nelecs):
            pow0 = data_freq[elec,...]
            R_trials, p_trials = np.array([]), np.array([])
            for t0, t1 in combinations(np.arange(ntrials), 2):
                R, p = stats.pearsonr(pow0[:,t0],pow0[:,t1])
                D = 1 - R # <<<<<<< HERE TO CHANGE FOR DISTANCE COMPUTATIONS
                R_trials = np.vstack((R_trials,D)) if np.size(R_trials) else D
                p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
            R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
            p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #R_freq = np.arctanh(R_freq)
        print(su,cond,freq,'TPSim',R_freq.shape, p_freq.shape,'initial data',pow_data.shape)
        dict_ = {'tps':R_freq, 'pval':p_freq,'label':mat['labels'],
                           'channel':mat['channels'], 'xyz':mat['xyz']}
        np.savez(savename.format(exp,su,cond,freq,step),**dict_)

for su,cond in product(subjects,conds):
    tpsim_by_cond(su,cond)
# Parallel(n_jobs=-1)(delayed(
#     tpsim_by_cond)(su,cond) for su,cond in product(subjects,conds))

-> Olfacto loaded
PIRJ low (6, 18, 64, 7) ['IFG' 'OFC_olf' 'PHG' 'aHC']
PIRJ low theta TPSim (18, 21) (18, 21) initial data (6, 18, 30, 7)
PIRJ mid (6, 18, 64, 4) ['IFG' 'OFC_olf' 'PHG' 'aHC']
PIRJ mid theta TPSim (18, 6) (18, 6) initial data (6, 18, 30, 4)
PIRJ high (6, 18, 64, 7) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
PIRJ high theta TPSim (18, 21) (18, 21) initial data (6, 18, 30, 7)
CHAF low (6, 61, 64, 3) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF low theta TPSim (61, 3) (61, 3) initial data (6, 61, 30, 3)
CHAF mid (6, 61, 64, 2) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF mid theta TPSim (61, 1) (61, 1) initial data (6, 61, 30, 2)
CHAF high (6, 61, 64, 5) ['ACC' 'MFG' 'OFC_olf' 'SFG']
CHAF high theta TPSim (61, 10) (61, 10) initial data (6, 61, 30, 5)
FERJ low (6, 32, 64, 7) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
FERJ low theta TPSim (32, 21) (32, 21) initial data (6, 32, 30, 7)
FERJ mid (6, 32, 64, 2) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
FERJ mid theta TPSim (32, 1) (32, 1) initial data (6, 32,

### Compute similarity BTW odors through learning

In [23]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect
Compute TPSim for all ODORS whatever the memory perf group
ALL ODORS TOGETHER // EARLY MID LATE GROUPS OF TRIALS
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_file = join(path_data, '{}_odor_{}_bipo_all_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
###############################################################################
freq = 'theta'
to_take = [17,47]

def tpsim_by_cond_learn(data):
    nelecs, npts, ntrials = data.shape
    ncomb = len([a for a,_ in combinations(np.arange(ntrials), 2)])
    tps_trials = np.empty((nelecs,ncomb))
    for e,elec in enumerate(range(nelecs)):
        pow0 = data[elec,...]
        c = 0
        for t0, t1 in combinations(np.arange(ntrials), 2):
            tps_trials[e,c], _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            c += 1
    tps_trials = 1 - np.arctanh(tps_trials)
    return tps_trials

for su in odor_list_su('Enc'):
    early_t, mid_t, late_t = [], [], []
    for od in odor_list_su('Enc')[su]:
        mat = np.load(pow_file.format(su,od),allow_pickle=True)
        pow_ = mat['xpow'][1,:,to_take[0]:to_take[1],:]
        thr = int(pow_.shape[-1]/3)
        print(pow_.shape,thr)
        if thr == 0:
            if pow_.shape[-1] >= 1:
                early_t.append(pow_[...,:thr+1])
            if pow_.shape[-1] == 2:
                mid_t.append(pow_[...,thr+1:])
        if thr >= 1:
            #if pow_.shape[-1] == 4:
            #    early_t.append(pow_[...,:thr])
            #    mid_t.append(pow_[...,thr:3])
            #    late_t.append(pow_[...,3:])
            #if pow_.shape[-1] == 5:
            #    early_t.append(pow_[...,:2])
            #    mid_t.append(pow_[...,2:4])
            #    late_t.append(pow_[...,4:])
            #if (pow_.shape[-1] >= 6) or (pow_.shape[-1]==3):
            early_t.append(pow_[...,:thr])
            mid_t.append(pow_[...,thr:(2*thr)])
            late_t.append(pow_[...,(2*thr):])
    print('early odors list', [x.shape for x in early_t])
    print('mid odors list', [x.shape for x in mid_t])
    print('late odors list', [x.shape for x in late_t])

    concat_E = np.concatenate(early_t, axis=-1)
    concat_M = np.concatenate(mid_t, axis=-1)
    concat_L = np.concatenate(late_t, axis=-1)
    tps_E = tpsim_by_cond_learn(concat_E)
    tps_M = tpsim_by_cond_learn(concat_M)
    tps_L = tpsim_by_cond_learn(concat_L)
    
    dict_t = {}
    dict_t['tps_0'], dict_t['tps_1'], dict_t['tps_2'] = tps_E, tps_M, tps_L
    dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
    np.savez(savename.format(su,freq,'3gr'),**dict_t)

-> Olfacto loaded
(61, 30, 4) 1
(61, 30, 3) 1
(61, 30, 5) 1
(61, 30, 3) 1
(61, 30, 3) 1
early odors list [(61, 30, 1), (61, 30, 1), (61, 30, 1), (61, 30, 1), (61, 30, 1)]
mid odors list [(61, 30, 1), (61, 30, 1), (61, 30, 1), (61, 30, 1), (61, 30, 1)]
late odors list [(61, 30, 2), (61, 30, 1), (61, 30, 3), (61, 30, 1), (61, 30, 1)]
(27, 30, 2) 0
(27, 30, 5) 1
(27, 30, 4) 1
(27, 30, 4) 1
(27, 30, 6) 2
(27, 30, 11) 3
(27, 30, 16) 5
(27, 30, 7) 2
early odors list [(27, 30, 1), (27, 30, 1), (27, 30, 1), (27, 30, 1), (27, 30, 2), (27, 30, 3), (27, 30, 5), (27, 30, 2)]
mid odors list [(27, 30, 1), (27, 30, 1), (27, 30, 1), (27, 30, 1), (27, 30, 2), (27, 30, 3), (27, 30, 5), (27, 30, 2)]
late odors list [(27, 30, 3), (27, 30, 2), (27, 30, 2), (27, 30, 2), (27, 30, 5), (27, 30, 6), (27, 30, 3)]
(18, 30, 4) 1
(18, 30, 6) 2
(18, 30, 7) 2
(18, 30, 5) 1
early odors list [(18, 30, 1), (18, 30, 2), (18, 30, 2), (18, 30, 1)]
mid odors list [(18, 30, 1), (18, 30, 2), (18, 30, 2), (18, 30, 1)]
late odo

In [27]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect
Compute TPSim for all ODORS whatever the memory perf group
ALL ODORS TOGETHER // EARLY AND LATE GROUPS OF TRIALS
"""
###############################################################################
st = study('Olfacto')
conds = ['low','mid','high']
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_file = join(path_data, '{}_odor_{}_bipo_all_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
###############################################################################
freq = 'theta'
to_take = [17,47]

def tpsim_by_cond_learn(data):
    nelecs, npts, ntrials = data.shape
    ncomb = len([a for a,_ in combinations(np.arange(ntrials), 2)])
    tps_trials = np.empty((nelecs,ncomb))
    for e,elec in enumerate(range(nelecs)):
        pow0 = data[elec,...]
        c = 0
        for t0, t1 in combinations(np.arange(ntrials), 2):
            tps_trials[e,c], _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            c += 1
    tps_trials = 1 - np.arctanh(tps_trials)
    return tps_trials

for su in odor_list_su('Enc'):
    early_t, late_t = [], []
    for od in odor_list_su('Enc')[su]:
        mat = np.load(pow_file.format(su,od),allow_pickle=True)
        pow_ = mat['xpow'][1,:,to_take[0]:to_take[1],:]
        thr = int(pow_.shape[-1]/2)
        if thr == 0:
            early_t.append(pow_[...,:thr+1])
        else:
            early_t.append(pow_[...,:thr]), late_t.append(pow_[...,thr:])
    print('early odors list', [x.shape for x in early_t])
    print('late odors list', [x.shape for x in late_t])

    concat_E = np.concatenate(early_t, axis=-1)
    concat_L = np.concatenate(late_t, axis=-1)
    tps_E = tpsim_by_cond_learn(concat_E)
    tps_L = tpsim_by_cond_learn(concat_L)
    
    dict_t = {}
    dict_t['tps_0'], dict_t['tps_1'] = tps_E, tps_L
    dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
    np.savez(savename.format(su,freq,'2gr'),**dict_t)

-> Olfacto loaded
early odors list [(61, 30, 2), (61, 30, 1), (61, 30, 2), (61, 30, 1), (61, 30, 1)]
late odors list [(61, 30, 2), (61, 30, 2), (61, 30, 3), (61, 30, 2), (61, 30, 2)]
early odors list [(27, 30, 1), (27, 30, 2), (27, 30, 2), (27, 30, 2), (27, 30, 3), (27, 30, 5), (27, 30, 8), (27, 30, 3)]
late odors list [(27, 30, 1), (27, 30, 3), (27, 30, 2), (27, 30, 2), (27, 30, 3), (27, 30, 6), (27, 30, 8), (27, 30, 4)]
early odors list [(18, 30, 2), (18, 30, 3), (18, 30, 3), (18, 30, 2)]
late odors list [(18, 30, 2), (18, 30, 3), (18, 30, 4), (18, 30, 3)]
early odors list [(39, 30, 1), (39, 30, 1), (39, 30, 1), (39, 30, 1), (39, 30, 1), (39, 30, 1), (39, 30, 1), (39, 30, 2)]
late odors list [(39, 30, 2), (39, 30, 1), (39, 30, 2), (39, 30, 1), (39, 30, 2), (39, 30, 2), (39, 30, 1), (39, 30, 2)]
early odors list [(53, 30, 3), (53, 30, 2), (53, 30, 3), (53, 30, 5), (53, 30, 1), (53, 30, 2), (53, 30, 1), (53, 30, 1)]
late odors list [(53, 30, 4), (53, 30, 2), (53, 30, 3), (53, 30, 5), (

In [20]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect for WITHIN COMPARISONS
Compute TPSim for all ODORS whatever the memory perf group 
ALL ODORS TOGETHER // EARLY AND LATE GROUPS OF TRIALS
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_file = join(path_data, '{}_odor_{}_bipo_all_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/by_odor_wth_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_wth_{}_{}_2gr.npz')
###############################################################################
f,freq = 1,'theta'
to_take = [17,47]
reps = ['Enc'] #Ret, Enc
for rep in reps:
    if not exists(pathsave.format(rep)):
        makedirs(pathsave.format(rep))

odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

def tpsim_by_odor(data):
    nelecs,npts,ntrials = data.shape
    tps_wth = np.array([])
    for elec in range(nelecs):
        pow0 = data[elec,:]
        tps_trials = np.array([])
        for t0, t1 in combinations(np.arange(ntrials), 2):
            R, _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            tps_trials = np.vstack((tps_trials,R)) if np.size(tps_trials) else R
        tps_wth = np.vstack((tps_wth,tps_trials.T)) if np.size(tps_wth) else tps_trials.T
    #tps_wth = np.arctanh(tps_wth)
    print(su,odor,freq,'R',tps_wth.shape, 'initial data',data.shape)
    return tps_wth

def compute_save_tpsim_wth_step(exp,su,odor,f,freq):
    mat = np.load(pow_path.format(su,odor,rep[0]),allow_pickle=True)
    pow_data = mat['xpow'][f,:,to_take[0]:to_take[1],:]
    print(su,odor,pow_data.shape)
    
    #separate data in two groups
    thr = int(pow_data.shape[-1]/2)
    if thr != 0:
        tps_E = tpsim_by_odor(pow_data[...,:thr])
        tps_L = tpsim_by_odor(pow_data[...,thr:])
    
        dict_t = {}
        dict_t['tps_0'], dict_t['tps_1'] = tps_E, tps_L
        dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
        np.savez(savename.format(su,freq,odor),**dict_t)

for rep in reps:
    for su in odors_su:
        for odor in odors_su[su]:
            compute_save_tpsim_wth_step(rep,su,odor,f,freq)

-> Olfacto loaded
CHAF 5 (61, 30, 1)
CHAF 7 (61, 30, 4)
CHAF 7 theta R (61, 1) initial data (61, 30, 2)
CHAF 7 theta R (61, 1) initial data (61, 30, 2)
CHAF 8 (61, 30, 3)
CHAF 8 theta R (0,) initial data (61, 30, 1)
CHAF 8 theta R (61, 1) initial data (61, 30, 2)
CHAF 9 (61, 30, 5)
CHAF 9 theta R (61, 1) initial data (61, 30, 2)
CHAF 9 theta R (61, 3) initial data (61, 30, 3)
CHAF 1 (61, 30, 1)
CHAF 2 (61, 30, 3)
CHAF 2 theta R (0,) initial data (61, 30, 1)
CHAF 2 theta R (61, 1) initial data (61, 30, 2)
CHAF 3 (61, 30, 3)
CHAF 3 theta R (0,) initial data (61, 30, 1)
CHAF 3 theta R (61, 1) initial data (61, 30, 2)
CHAF 4 (61, 30, 1)
LEFC 1 (27, 30, 2)
LEFC 1 theta R (0,) initial data (27, 30, 1)
LEFC 1 theta R (0,) initial data (27, 30, 1)
LEFC 2 (27, 30, 5)
LEFC 2 theta R (27, 1) initial data (27, 30, 2)
LEFC 2 theta R (27, 3) initial data (27, 30, 3)
LEFC 3 (27, 30, 4)
LEFC 3 theta R (27, 1) initial data (27, 30, 2)
LEFC 3 theta R (27, 1) initial data (27, 30, 2)
LEFC 4 (27, 30, 4)
L

In [23]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect for BETWEEN COMPARISONS
Compute TPSim for all ODORS whatever the memory perf group 
ALL ODORS TOGETHER // EARLY AND LATE GROUPS OF TRIALS
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_file = join(path_data, '{}_odor_{}_bipo_all_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/by_odor_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}_2gr.npz')
###############################################################################
f,freq = 1,'theta'
to_take = [17,47]
reps = ['Enc'] #Ret, Enc
for rep in reps:
    if not exists(pathsave.format(rep)):
        makedirs(pathsave.format(rep))

odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

def tpsim_by_odor(data):
    nelecs,npts,ntrials = data.shape
    tps_wth = np.array([])
    for elec in range(nelecs):
        pow0 = data[elec,:]
        tps_trials = np.array([])
        for t0, t1 in combinations(np.arange(ntrials), 2):
            R, _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            tps_trials = np.vstack((tps_trials,R)) if np.size(tps_trials) else R
        tps_wth = np.vstack((tps_wth,tps_trials.T)) if np.size(tps_wth) else tps_trials.T
    #tps_wth = np.arctanh(tps_wth)
    print(su,odor,freq,'R',tps_wth.shape, 'initial data',data.shape)
    return tps_wth

def separate_2gr(pow_data):
    thr = int(pow_data.shape[-1]/2)
    all_files = []
    if thr != 0:
        all_files.append(pow_data[...,:thr])
        all_files.append(pow_data[...,thr:])
    return all_files

for rep in reps:
    for su in odors_su:
        for o1 in odors_su[su]:
            o2 = [o for o in odors_su[su] if o != o1]
            mat_o1 = np.load(pow_file.format(su,o1,exp[0]),allow_pickle=True)
            list_o1 = separate_2gr(mat_o1['xpow'][f,:,17:47,:]) #nelecs,npts,ntrials
            if len(list_o1) > 0: #at least 2 repetitions of o1
                list_o2 = []
                for o in o2:
                    pow_ = np.load(pow_file.format(su,o,exp[0],freq))['xpow'][f,:,17:47,:]
                    pow_sep = separate_2gr(pow_)
                    if len(pow_sep) > 0:
                        list_o2.append(separate_2gr(pow_))
                o2_E = np.concatenate([x[0] for x in list_o2],axis=-1)
                o2_L = np.concatenate([x[1] for x in list_o2],axis=-1)
                tpsE = tpsim_btw_odors_by_odor(list_o1[0], o2_E)
                tpsL = tpsim_btw_odors_by_odor(list_o1[1], o2_L)
                mat = {'tps_0':tpsE, 'tps_1':tpsL,'label':mat_o1['labels'],
                               'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
                np.savez(savename.format(su,str(o1),freq),**mat)

-> Olfacto loaded


In [22]:
def tpsim_btw_odors_by_odor(pow_o1,pow_o2):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2)
    """
    
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.pearsonr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    return R_freq
    

In [26]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect
Compute TPSim for all ODORS BY memory perf group
EARLY AND LATE GROUPS OF TRIALS
"""
###############################################################################
st = study('Olfacto')
conds,exp = ['low','mid','high'],'Enc'
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
pow_file = join(path_data, '{}_odor_{}_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
###############################################################################
freq, meth = 'theta', '2gr' #groups here refer to time groups early late
to_take = [17,47]

def tpsim_by_cond_learn(data):
    nelecs, npts, ntrials = data.shape
    ncomb = len([a for a,_ in combinations(np.arange(ntrials), 2)])
    tps_trials = np.empty((nelecs,ncomb))
    for e,elec in enumerate(range(nelecs)):
        pow0 = data[elec,...]
        c = 0
        for t0, t1 in combinations(np.arange(ntrials), 2):
            tps_trials[e,c], _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            c += 1
    tps_trials = 1 - tps_trials
    return tps_trials

for su in odor_list_su('Enc'):
    for cond in conds:
        mat = np.load(pow_file.format(su,cond),allow_pickle=True)
        pow_ = mat['xpow'][1,:,to_take[0]:to_take[1],:]
        if meth == '2gr':
            thr = int(pow_.shape[-1]/2)
            early_c, late_c = pow_[...,:thr], pow_[...,thr:]
            print('early ', early_c.shape)
            print('late ', late_c.shape)

            tps_E = tpsim_by_cond_learn(early_c)
            tps_L = tpsim_by_cond_learn(late_c)

            dict_t = {}
            dict_t['tps_0'], dict_t['tps_1'] = tps_E, tps_L
            dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
            np.savez(savename.format(su,freq,cond,'2gr'),**dict_t)
        
        elif meth =='3gr':
            thr = int(pow_.shape[-1]/3)
            print(pow_.shape,thr)
            if thr == 0:
                if pow_.shape[-1] >= 1:
                    early_c = pow_[...,:thr+1]
                if pow_.shape[-1] == 2:
                    mid_c = pow_[...,thr+1:]
            if thr >= 1:
                early_c = pow_[...,:thr]
                mid_c = pow_[...,thr:(2*thr)]
                late_c = pow_[...,(2*thr):]
            print('early odors list', [x.shape for x in early_c])
            print('mid odors list', [x.shape for x in mid_c])
            print('late odors list', [x.shape for x in late_c])

            tps_E = tpsim_by_cond_learn(early_c)
            tps_M = tpsim_by_cond_learn(mid_c)
            tps_L = tpsim_by_cond_learn(late_c)
    
            dict_t = {}
            dict_t['tps_0'], dict_t['tps_1'], dict_t['tps_2'] = tps_E, tps_M, tps_L
            dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
            np.savez(savename.format(su,freq,cond,'3gr'),**dict_t)

-> Olfacto loaded
early  (61, 30, 2)
late  (61, 30, 3)
early  (61, 30, 2)
late  (61, 30, 2)
early  (61, 30, 6)
late  (61, 30, 6)
early  (27, 30, 11)
late  (27, 30, 12)
early  (27, 30, 7)
late  (27, 30, 8)
early  (27, 30, 8)
late  (27, 30, 9)
early  (18, 30, 7)
late  (18, 30, 7)
early  (18, 30, 4)
late  (18, 30, 5)
early  (18, 30, 7)
late  (18, 30, 7)
early  (39, 30, 4)
late  (39, 30, 5)
early  (39, 30, 2)
late  (39, 30, 2)
early  (39, 30, 4)
late  (39, 30, 5)
early  (53, 30, 10)
late  (53, 30, 11)
early  (53, 30, 5)
late  (53, 30, 5)
early  (53, 30, 4)
late  (53, 30, 5)
early  (32, 30, 8)
late  (32, 30, 8)
early  (32, 30, 3)
late  (32, 30, 3)
early  (32, 30, 4)
late  (32, 30, 5)


In [ ]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect
Compute TPSim for all ODORS whatever the memory perf group
ALL ODORS TOGETHER // ALL trials INDIVIDUALLY no groups
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
pow_file = join(path_data, '{}_odor_{}_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
###############################################################################
freq = 'theta'
to_take = [17,47]

def tpsim_by_cond_learn(data):
    nelecs, npts, ntrials = data.shape
    ncomb = len([a for a,_ in combinations(np.arange(ntrials), 2)])
    tps_trials = np.empty((nelecs,ncomb))
    for e,elec in enumerate(range(nelecs)):
        pow0 = data[elec,...]
        c = 0
        for t0, t1 in combinations(np.arange(ntrials), 2):
            tps_trials[e,c], _ = stats.pearsonr(pow0[:,t0],pow0[:,t1])
            c += 1
    tps_trials = 1 - np.arctanh(tps_trials)
    return tps_trials

for su in odor_list_su('Ret'):
    all_odors = []
    for od in odor_list_su('Ret')[su]:
        mat = np.load(pow_file.format(su,od))
        pow_ = mat['xpow'][1,:,to_take[0]:to_take[1],:]
        all_odors.append(pow_)
        
    max_trials = np.max([x.shape[-1] for x in all_odors])
    list_tpsim = []
    for t in range(max_trials):
        list_t = [x[:,:,t][np.newaxis] for x in all_odors if x.shape[-1] > t]
        if len(list_t) > 1 :
            concat_od = np.concatenate(list_t, axis=0).swapaxes(0,-1).swapaxes(0,1)
            tps_t = tpsim_by_cond_learn(concat_od)
            list_tpsim.append(tps_t)

    print(su,freq,max_trials,[x.shape for x in list_tpsim])
    dict_t = {}
    for n in range(len(list_tpsim)):
        dict_t['tps_'+str(n)] = list_tpsim[n]
    dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
    np.savez(savename.format(su,freq,cond),**dict_t)

In [ ]:
def tpsim_btw_odors(array1,array2):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    array1, array2 : array
        shape nelecs, npts, ntrials (same shape of elecs and npts)

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between 2 two arrays
        (n_combinations of trials btw array1 and array2)
    """
    tpsim_btw = np.array([])
    for elec in range(array1.shape[0]):
        pow1, pow2 = array1[elec], array2[elec]
        R_trials = np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.pearsonr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
        tpsim_btw = np.vstack((tpsim_btw,R_trials.T)) if np.size(tpsim_btw) else R_trials.T
    tpsim_btw = 1 - np.arctanh(tpsim_btw)
    return tpsim_btw

In [ ]:
from utils import odor_list_su, odor_groups_3wgth
"""
LEARNING effect
Compute TPSim for all ODORS whatever the memory perf group
SIMILARITY BTW ALL ODORS across trials
trial1 with trial2 / trial2 with trial3 / trial 3 with rest of trials
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_file = join(path_data, '{}_odor_{}_bipo_all_E_6freqs.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
savename = join(pathsave,'TPS_pears_learn_{}_btw_{}_{}.npz')
###############################################################################
if not exists(pathsave.format(exp)):
    makedirs(pathsave.format(exp))
###############################################################################
freq = 'theta'
to_take = [17,47]

for su in odor_list_su('Ret'):
    all_odors = []
    for od in odor_list_su('Ret')[su]:
        mat = np.load(pow_file.format(su,od),allow_pickle=True)
        pow_ = mat['xpow'][1,:,to_take[0]:to_take[1],:]
        all_odors.append(pow_)
    print([x.shape for x in all_odors])
    
    list_tpsim = []
    for t in range(3):
        list0 = [x[:,:,t][...,np.newaxis] for x in all_odors if x.shape[-1] > t]
        if t == 2 :
            list1 = [x[:,:,t+1:] for x in all_odors if x.shape[-1] > t+1]
        else :
            list1 = [x[:,:,t+1][...,np.newaxis] for x in all_odors if x.shape[-1] > t+1]
        concat_1 = np.concatenate(list0, axis=-1)
        concat_2 = np.concatenate(list1, axis=-1)
        tps_t = tpsim_btw_odors(concat_1,concat_2)
        list_tpsim.append(tps_t)
        
    print(su,freq,[x.shape for x in list_tpsim])
    dict_t = {}
    for n in range(len(list_tpsim)):
        dict_t['tps_'+str(n)] = list_tpsim[n]
    dict_t['label'], dict_t['channel'], dict_t['xyz'] = mat['labels'],mat['channels'], mat['xyz']
    np.savez(savename.format(su,freq,'btw_trials'),**dict_t)